# Database Creation

This notebook creates a SQLite database (`transactions.db`) from the CSV file (`data/data.csv`). It performs data cleaning, transformation, and loads the data into a properly structured SQLite database.

## Overview

The process involves:
1. **Importing libraries** - Setting up required Python packages
2. **Configuration** - Setting up file paths
3. **Reading CSV** - Loading transaction data from CSV file
4. **Data Cleaning** - Removing duplicates and standardizing column names
5. **Data Transformation** - Renaming columns, parsing dates, and adding derived fields
6. **Database Creation** - Creating SQLite database with explicit schema
7. **Validation** - Testing the created database


## Step 1: Import Required Libraries

Import necessary libraries for data processing and database operations:
- `pandas` - For data manipulation and CSV reading
- `sqlite3` - For SQLite database operations
- `sqlalchemy` - For database engine creation and type definitions
- `os` - For file path operations


In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, types
import os

## Step 2: Configure File Paths

Set up the paths for the input CSV file and output SQLite database. The script automatically detects whether it's running from the `scripts` directory or the project root, and adjusts paths accordingly.


In [3]:
# ---------- CONFIG ----------
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'scripts':
    # If running from scripts directory, go up one level
    project_root = os.path.dirname(current_dir)
else:
    # If running from project root, use current directory
    project_root = current_dir

csv_path = os.path.join(project_root, "data", "data.csv")
db_path = os.path.join(project_root, "database", "transactions.db")
table_name = "transactions"
# ----------------------------

## Step 3: Read CSV File

Load the transaction data from the CSV file into a pandas DataFrame. This will display the raw data structure with all columns (client_id, bank_id, account_id, transaction_id, transaction_date, description, amount, category, merchant).


In [4]:
# Read CSV
df = pd.read_csv(csv_path)

df

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant
0,6,1,1,54,31/07/2023 0:00,CLOC Advance,6.286,Shops,NaN
1,6,1,1,27,31/07/2023 0:00,CLOC Advance,6.286,Shops,NaN
2,6,1,1,11,01/08/2023 0:00,CLOC Advance,2.268,Shops,NaN
3,28,1,1,108,25/07/2023 0:00,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,59.100,Shops,1INFINITE
4,28,1,1,136,14/08/2023 0:00,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,4.924,Shops,1INFINITE
...,...,...,...,...,...,...,...,...,...
257058,880,488,557,75841,11/09/2023 0:00,BKOFAMMaryse Hemant ATM 09/09 #3168 WITHDRWL M...,-128.000,ATM,NaN
257059,880,488,557,69194,11/09/2023 0:00,BKOFAMMaryse Hemant ATM 09/10 #3168 WITHDRWL M...,-16.000,ATM,NaN
257060,880,489,558,99189,13/06/2023 0:00,ATM WITHDRAWAL AUTHORIZED ON 1036 MOUNT HOLLY ...,-60.000,ATM,NaN
257061,880,489,558,56121,18/07/2023 0:00,NON-WF ATM WITHDRAWAL AUTHORIZED ON 1036 PENRO...,-4.000,ATM,NaN


## Step 4: Clean Column Names

Clean and standardize column names by:
- Removing duplicate columns (if any)
- Stripping whitespace from column names
- Converting to lowercase
- Replacing spaces with underscores

This ensures consistent naming conventions for database compatibility.


In [5]:
df = df.loc[:, ~df.columns.duplicated()]  # remove duplicates
df.columns = [c.strip().replace(" ", "_").lower() for c in df.columns]

df

,clnt_id,bank_id,acc_id,txn_id,txn_date,desc,amt,cat,merchant
0,6,1,1,54,31/07/2023 0:00,CLOC Advance,6.286,Shops,NaN
1,6,1,1,27,31/07/2023 0:00,CLOC Advance,6.286,Shops,NaN
2,6,1,1,11,01/08/2023 0:00,CLOC Advance,2.268,Shops,NaN
3,28,1,1,108,25/07/2023 0:00,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,59.100,Shops,1INFINITE
4,28,1,1,136,14/08/2023 0:00,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,4.924,Shops,1INFINITE
...,...,...,...,...,...,...,...,...,...
257058,880,488,557,75841,11/09/2023 0:00,BKOFAMMaryse Hemant ATM 09/09 #3168 WITHDRWL M...,-128.000,ATM,NaN
257059,880,488,557,69194,11/09/2023 0:00,BKOFAMMaryse Hemant ATM 09/10 #3168 WITHDRWL M...,-16.000,ATM,NaN
257060,880,489,558,99189,13/06/2023 0:00,ATM WITHDRAWAL AUTHORIZED ON 1036 MOUNT HOLLY ...,-60.000,ATM,NaN
257061,880,489,558,56121,18/07/2023 0:00,NON-WF ATM WITHDRAWAL AUTHORIZED ON 1036 PENRO...,-4.000,ATM,NaN


## Step 5: Transform Data

Perform key data transformations:
1. **Rename columns** - Map abbreviated column names to full, descriptive names (e.g., `clnt_id` → `client_id`, `txn_date` → `transaction_date`)
2. **Parse dates** - Convert transaction_date from string format to datetime objects using day-first format (DD/MM/YYYY)
3. **Add transaction_type** - Derive transaction type from amount:
   - `Credit` for positive amounts (money in)
   - `Debit` for negative amounts (money out)


In [6]:
rename_map = {
    "clnt_id": "client_id",
    "bank_id": "bank_id",
    "acc_id": "account_id",
    "txn_id": "transaction_id",
    "txn_date": "transaction_date",
    "desc": "description",
    "amt": "amount",
    "cat": "category",
    "merchant": "merchant",
}
df = df.rename(columns=rename_map)

# Parse transaction_date including time
df["transaction_date"] = pd.to_datetime(df["transaction_date"], dayfirst=True)

df["transaction_type"] = df["amount"].apply(lambda x: "Credit" if x > 0 else "Debit")

df

,client_id,bank_id,account_id,transaction_id,transaction_date,description,amount,category,merchant,transaction_type
0,6,1,1,54,2023-07-31,CLOC Advance,6.286,Shops,NaN,Credit
1,6,1,1,27,2023-07-31,CLOC Advance,6.286,Shops,NaN,Credit
2,6,1,1,11,2023-08-01,CLOC Advance,2.268,Shops,NaN,Credit
3,28,1,1,108,2023-07-25,1INFINITELOOP@ 07/25 #68 PMNT RCVD 1INFINITELO...,59.100,Shops,1INFINITE,Credit
4,28,1,1,136,2023-08-14,1INFINITELOOP@ 08/14 #68 PMNT RCVD 1INFINITELO...,4.924,Shops,1INFINITE,Credit
...,...,...,...,...,...,...,...,...,...,...
257058,880,488,557,75841,2023-09-11,BKOFAMMaryse Hemant ATM 09/09 #3168 WITHDRWL M...,-128.000,ATM,NaN,Debit
257059,880,488,557,69194,2023-09-11,BKOFAMMaryse Hemant ATM 09/10 #3168 WITHDRWL M...,-16.000,ATM,NaN,Debit
257060,880,489,558,99189,2023-06-13,ATM WITHDRAWAL AUTHORIZED ON 1036 MOUNT HOLLY ...,-60.000,ATM,NaN,Debit
257061,880,489,558,56121,2023-07-18,NON-WF ATM WITHDRAWAL AUTHORIZED ON 1036 PENRO...,-4.000,ATM,NaN,Debit


## Step 6: Create SQLite Database

Create the SQLite database using SQLAlchemy with:
- **Explicit schema definition** - Define appropriate data types for each column (Integer, Float, Text, DateTime)
- **Table creation** - Create the `transactions` table with the cleaned and transformed data
- **Replace mode** - Overwrite existing table if it already exists

This ensures type safety and proper database structure for efficient querying.


In [7]:
# 3. Create SQLAlchemy engine for SQLite
engine = create_engine(f"sqlite:///{db_path}")

# 4. Define SQLite datatypes for each column
column_types = {
    "client_id": types.Integer(),
    "bank_id": types.Integer(),
    "account_id": types.Integer(),
    "transaction_id": types.Integer(),
    "transaction_date": types.DateTime(),
    "transaction_type": types.Text(),
    "description": types.Text(),
    "amount": types.Float(),
    "category": types.Text(),
    "merchant": types.Text(),
}

# 5. Write DataFrame to SQLite with explicit schema
df.to_sql(
    table_name,
    engine,
    if_exists="replace",   # replace table if already exists
    index=False,
    dtype=column_types     # apply schema
)

print("CSV successfully imported into SQLite with explicit datatypes!")

CSV successfully imported into SQLite with explicit datatypes!


## Step 7: Validate Database

Run validation tests to verify the database was created correctly:
1. **Table existence** - Verify the `transactions` table exists
2. **Data population** - Confirm the table contains data (row count > 0)
3. **Schema validation** - Check that all required columns exist with correct data types
4. **Sample query** - Execute a test query to verify data integrity

All tests should pass before the database is considered ready for use.


In [8]:
import sqlite3
import pandas as pd
import os

current_dir = os.getcwd()
if os.path.basename(current_dir) == 'scripts':
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

db_path = os.path.join(project_root, "database", "transactions.db")
table = "transactions"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def run_test(test_name, query, expected=None, check_fn=None):
    print(f"\n=== {test_name} ===")
    result = cursor.execute(query).fetchall()
    print("Result:", result)

    if expected is not None:
        if result == expected:
            print("✅ PASSED")
        else:
            print("❌ FAILED — expected:", expected)
    elif check_fn:
        if check_fn(result):
            print("✅ PASSED")
        else:
            print("❌ FAILED")
    else:
        print("⚠️ No expected value provided")

# 1. Test that table exists
run_test(
    "Test 1: Table exists",
    f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}';",
    expected=[(table,)]
)

# 2. Test: Row count > 0
run_test(
    "Test 2: Table is not empty",
    f"SELECT COUNT(*) FROM {table};",
    check_fn=lambda r: r[0][0] > 0
)

# 3. Test: Required columns exist
required_cols = ["client_id", "bank_id", "account_id", "transaction_id",
                 "transaction_date", "transaction_type", "description", "amount", "category", "merchant"]

run_test(
    "Test 3: Required columns exist",
    f"PRAGMA table_info({table});",
    check_fn=lambda r: all(col in [row[1] for row in r] for col in required_cols)
)

# 4. Test: Sample query - check first 5 rows load correctly
run_test(
    "Test 4: Fetch Details",
    f"SELECT * FROM {table} WHERE client_id = 809 and bank_id = 1 and account_id = 1 and transaction_id = 1;"
)


conn.close()



=== Test 1: Table exists ===
Result: [('transactions',)]
✅ PASSED

=== Test 2: Table is not empty ===
Result: [(257063,)]
✅ PASSED

=== Test 3: Required columns exist ===
Result: [(0, 'client_id', 'INTEGER', 0, None, 0), (1, 'bank_id', 'INTEGER', 0, None, 0), (2, 'account_id', 'INTEGER', 0, None, 0), (3, 'transaction_id', 'INTEGER', 0, None, 0), (4, 'transaction_date', 'DATETIME', 0, None, 0), (5, 'description', 'TEXT', 0, None, 0), (6, 'amount', 'FLOAT', 0, None, 0), (7, 'category', 'TEXT', 0, None, 0), (8, 'merchant', 'TEXT', 0, None, 0), (9, 'transaction_type', 'TEXT', 0, None, 0)]
✅ PASSED

=== Test 4: Fetch Details ===
Result: [(809, 1, 1, 1, '2023-07-24 23:00:00.000000', 'Visa Direct Apple Cash Inst 07/23 Card 4851', 2.3, 'Shops', 'Apple', 'Credit')]
⚠️ No expected value provided
